# Species Bags in Species-collectors Networks
#### Date: 
#### Author: Pedro C. de Siracusa
---

In [1]:
import numpy as np
import pandas as pd
import networkx as nx

## Preparing Data

### Loading the UB Herbarium dataset

In [2]:
dsetPath = '/home/pedro/datasets/ub_herbarium/occurrence.txt'

In [3]:
cols = ['recordedBy', 'scientificName', 'family', 'genus', 'species','taxonRank', 
        'stateProvince', 'locality', 'municipality', 'occurrenceRemarks',
        'eventDate', 'identifiedBy']

occs = pd.read_table(dsetPath, delimiter='\t', usecols=cols, low_memory=False)

I'll only use data with taxonomic resolution of species

In [4]:
occs.dropna(subset=['recordedBy','scientificName'], inplace=True)
occs = occs[occs['taxonRank']=='SPECIES']

In [5]:
occs.head()

,recordedBy,occurrenceRemarks,eventDate,stateProvince,municipality,locality,identifiedBy,scientificName,family,genus,taxonRank,species
0,"Irwin, HS","Ascending subshrub 0,3m. Fruit gray-green; Cer...",1972-01-16T01:00Z,Goiás,NaN,"Serra dos Pireneus, ca. 20km E of Pirenópolis",NaN,Annona monticola Mart.,Annonaceae,Annona,SPECIES,Annona monticola
1,"Ratter, JA; et al.",Coppice branches sprouting from the stump of a...,1976-06-30T01:00Z,Minas Gerais,NaN,"Near Pandeiros, ca. 30.0 km W of Januária",Flora do Brasil,Myracrodruon urundeuva Allem.,Anacardiaceae,Myracrodruon,SPECIES,Myracrodruon urundeuva
2,"Heringer, EP",NaN,1954-06-05T01:00Z,Minas Gerais,NaN,Fazenda do Rasgão. Terreno de cultura,Flora do Brasil,Myracrodruon urundeuva Allem.,Anacardiaceae,Myracrodruon,SPECIES,Myracrodruon urundeuva
3,"Coelho, JP","Pouco frequente, sem folhas nesse período; Árv...",1964-10-15T01:00Z,Minas Gerais,NaN,IPEACO- Sete lagoas,Flora do Brasil,Myracrodruon urundeuva Allem.,Anacardiaceae,Myracrodruon,SPECIES,Myracrodruon urundeuva
4,"Eiten, G; Eiten, LT","Tree 5.0 m. tall, 11.0 cm d.b.h. With young fr...",1963-08-17T01:00Z,Maranhão,Loreto,"Ilha de Balsas region, between the Balsas and ...",Flora do Brasil,Myracrodruon urundeuva Allem.,Anacardiaceae,Myracrodruon,SPECIES,Myracrodruon urundeuva


In [6]:
occs['taxonRank'].value_counts()

SPECIES    140763
Name: taxonRank, dtype: int64

In [7]:
"Total num of records: "+str(occs.shape[0])

'Total num of records: 140763'

### Loading previously cleaned collectors identities

In [8]:
from modules.cleaning.names import read_NamesMap_fromJson, NamesAtomizer, namesFromString

# read a names map
nm = read_NamesMap_fromJson('./ub_names_map_2.json')

# atomizing names
atomizingOp = lambda x: namesFromString(x)
na = NamesAtomizer(atomizingOp)

occs['recordedBy_atomized'] = na.atomize(occs['recordedBy'])

# build names index
from modules.cleaning.names import getNamesIndexes

ni = getNamesIndexes(occs,'recordedBy_atomized',nm.getMap())

/home/pedro/datascience_biodiversity/notebooks/modules/cleaning/names.py:536: UserWarning: A names map was created without a normalization function!
  warn("A names map was created without a normalization function!")


---

## Building the SCN Network model

In [9]:
from modules.models.networks import SpeciesCollectorsNetwork

In [10]:
scn = SpeciesCollectorsNetwork(species=occs['scientificName'],
                               collectorsNames=occs['recordedBy_atomized'],
                               namesMap=nm,
                               weighted=True)

First we want to filter some nodes out of the network

In [11]:
nodesToRemove = ['etal','','incognito']
scn.remove_nodes_from(nodesToRemove)

And then add some attributes

In [12]:
# degree
nx.set_node_attributes(scn,'degree',nx.degree(scn))

# weighted degree
nx.set_node_attributes(scn,'degree_weighted',nx.degree(scn,weight='weight'))

---

## Species Bags

In a SCN model each collector is linked to a set of species she has ever collected, and conversely each species is linked to the set of collectors that have ever recorded it. I define a collector's **species bag** as an array of counts of species in her collections history. Formally a species bag $ \sigma$ of a collector $\mu$ is represented as a vector

$$ 
\sigma^{(\mu)} =  \begin{bmatrix}
\sigma_1^{(\mu)}, \sigma_2^{(\mu)}, \cdots, \sigma_n^{(\mu)}
\end{bmatrix}  \quad : \quad 
n = |S_{sp}|
$$

where $n$ is the length of the species set $S_{sp}$ and $\sigma^{(\mu)}_i$ is the total number of records of species $i$ by collector $\mu$.

The species bag for a collector can be calculated by using the `getSpeciesBag` method from the `SpeciesCollectorsNetwork` class. It returns a tuple containing an index with the species names and a *Scipy* sparse matrix containing the species bag vector itself. The order in which species names are placed in the index is the same as in the vector. To give an example let's retrieve the species bag from 'Munhoz,C.B.R.'.

In [21]:
i,m = scn.getSpeciesBag('munhoz,cbr')

We can unpack an ordered list with species counts by zipping both the index and the vector. I'll only retrieve the top-20 species.

In [26]:
sorted( list(zip( i, np.array(m.todense()).flatten() )), key=lambda x: x[1], reverse=True )[:20]

[('Syngonanthus nitens (Bong.) Ruhland', 30),
 ('Rhynchospora globosa (Kunth) Roem. & Schult.', 25),
 ('Xyris hymenachne Mart.', 23),
 ('Axonopus brasiliensis (Spreng.) Kuhlm.', 21),
 ('Trachypogon spicatus (L.f.) Kuntze', 20),
 ('Comanthera xeranthemoides (Bong.) L.R.Parra & Giul.', 19),
 ('Rhynchospora consanguinea (Kunth) Boeckeler', 19),
 ('Paepalanthus elongatus (Bong.) Körn.', 18),
 ('Xyris savanensis Miq.', 18),
 ('Curtia tenuifolia (Aubl.) Knobl.', 17),
 ('Lagenocarpus rigidus (Kunth) Nees', 17),
 ('Andropogon virgatus Desv. ex Ham.', 16),
 ('Hyptis linarioides Pohl ex Benth.', 16),
 ('Xyris schizachne Mart.', 16),
 ('Rhynchospora emaciata (Nees) Boeckeler', 15),
 ('Syngonanthus gracilis (Bong.) Ruhland', 15),
 ('Paspalum hyalinum Nees ex Trin.', 14),
 ('Trimezia juncifolia Klatt', 14),
 ('Chamaecrista desvauxii (Collad.) Killip', 13),
 ('Rhynchanthera grandiflora (Aubl.) DC.', 13)]

### Species Bags Similarity

We might want to compare collectors in terms of their species bags. We calculate the **Species Bag Similarity** between collectors $\mu_i$ and $\mu_j$ by computing the *Cosine Similarity* of their species bags vectors:

$$
sim(\sigma^{(\mu_i)},\sigma^{(\mu_j)}) =
\cos \theta_{\mu_i,\mu_j} \equiv
\frac{  \sigma^{(\mu_i)} \cdot \sigma^{(\mu_j)}  }{  ||\sigma^{(\mu_i)}||_2  ||\sigma^{(\mu_j)}||_2  }
$$

where the output ranges from [0,1].

The function below calculates species bags similarity. It takes two optional collectors names in `col1` and `col2` parameters, and output type varies depending on how many collectors names are given.

In [27]:
from sklearn.metrics.pairwise import cosine_similarity
import scipy

def calculateSimilarity( G, col1=None, col2=None ):
    """
    Returns
    -------
    If neither col1 and col2 are informed, a tuple ( colsList, csr_sparse_similarityMatrix )
    If only col1 is informed, a tuple ( colsList, similarityVector )
    If col1 and col2 are informed, a tuple ( [col1,col2], similarity )
    """
    colList, spList, m = G._speciesBag_matrix
    
    if col1 is not None:
        if col2 is not None:
            sim = cosine_similarity( scn.getSpeciesBag(col1)[1], scn.getSpeciesBag(col2)[1] )[0][0] 
            return ([col1,col2], sim)
        else:
            simVector = cosine_similarity( scn.getSpeciesBag(col1)[1], m )
            return ( colList, scipy.sparse.csr_matrix(simVector) )
        
    else:
        return( colList, scipy.sparse.csr_matrix(cosine_similarity(m)) )

First, if we wanted to simply retrieve the entire species bags similarity matrix we can simply call this function without passing collectors names. Note that all elements in the diagonal of the similarity matrix are ones.

In [29]:
i,m = calculateSimilarity(scn)

In [32]:
i[:10]

['abbas,b',
 'abdala,gc',
 'abdo,msa',
 'abdon',
 'abe,lm',
 'abrahim,ma',
 'abreu,gx',
 'abreu,i',
 'abreu,lcr',
 'abreu,mc']

In [31]:
m.todense() # m is a csr sparse matrix

matrix([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  1.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  1., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  1.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  1.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

We can also calculate a similarity vector for a given collector, which holds her similarity with all other collectors, including itself. Let's compute, for example, the similarity vector for 'munhoz,cbr'.

In [98]:
i,m = calculateSimilarity(scn,'munhoz,cbr')

In [101]:
m.todense()

matrix([[ 0.00730726,  0.        ,  0.00353577, ...,  0.        ,
          0.        ,  0.        ]])

Who are the top-10 collectors most similar to Munhoz,CBR?

In [105]:
sorted( list(zip(i, np.array(m.todense()).flatten())), key=lambda x:x[1], reverse=True)[:10]

[('munhoz,cbr', 0.99999999999999267),
 ('amaral,ag', 0.75137262358463819),
 ('eugenio,cuo', 0.72327560020983406),
 ('emmert,p', 0.71216702601217519),
 ('mello,trb', 0.69920571204778381),
 ('chagas,ur', 0.65072933682219702),
 ('silva,dp', 0.65038200328433593),
 ('cristine,a', 0.58353717801179739),
 ('correa,acg', 0.57466387100179006),
 ('bijos,nr', 0.56124589372476164)]

Last we can also compute the similarity for a pair of collectors:

In [106]:
i,score = calculateSimilarity(scn,'munhoz,cbr','proenca,ceb')
"Similarity score between {} and {}: {}".format(*i,score)

'Similarity score between munhoz,cbr and proenca,ceb: 0.32207714985042757'

### Filtering values from a similarity matrix

It is useful to keep a record only of the most significant similarities in the matrix. We can define a threshold and filter out all of these element below it. I'll exemplify using rows 0 to 10 and columns 164 to 173.

In [107]:
i,m = calculateSimilarity(scn)

In [108]:
m[0:10,164:174].todense()

matrix([[ 0.        ,  0.03328406,  0.        ,  0.        ,  0.        ,
          0.        ,  0.09622504,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.00784513,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.00286314,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.01921656,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.19245009,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.       

Now, let's filter it with a threshold of 0.05

In [109]:
thresh=0.05

m.data = np.where(m.data >= thresh, m.data, 0)
m.eliminate_zeros()

In [110]:
m[0:10,164:174].todense()

matrix([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.09622504,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.19245009,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.       

### Creating a projected SCN Network from the similarity matrix

The similarity matrix can be used to build a similarity graph. The similarity graph is a type of SCN projection on the collectors set. In order not to add self-loops, I will set all diagonal elements of the similarity matrix to zero.

In [155]:
collNames,m = calculateSimilarity(scn)
m.setdiag(0)

thresh=0.3
m.data = np.where(m.data >= thresh, m.data, 0)

m.eliminate_zeros()

In [156]:
G = nx.from_scipy_sparse_matrix(m)

In [157]:
# relabel nodes
namesMap = dict( (i,n) for i,n in enumerate(collNames) )
G = nx.relabel_nodes(G,namesMap)

Let's now copy the attributes from the SCN to the similarity graph

In [158]:
# degree
deg_dict = dict((n,d['degree']) for n,d in scn.getCollectorsNodes(data=True))
nx.set_node_attributes(G, 'degree', deg_dict)

# weighted degree
degw_dict = dict((n,d['degree_weighted']) for n,d in scn.getCollectorsNodes(data=True))
nx.set_node_attributes(G, 'degree_weighted', degw_dict)

In [160]:
nx.write_gexf(G,'./graph_similarity.gexf')

---

### Calculate matrix sparsity

In [166]:
calculateSparsity = lambda m: len(m.data)/np.prod(m.shape)

In [168]:
i,m = calculateSimilarity(scn)

In [197]:
from copy import deepcopy
m_t = deepcopy(m)
thresh=0.5
m_t.data = np.where(m_t.data >= thresh, m_t.data, 0)

m_t.eliminate_zeros()

In [198]:
calculateSparsity(m_t)

0.00080515889947774971

In [183]:
calculateSparsity(m)

0.033692832405074334

In [161]:
len(m.data)

67286

In [165]:
len(m.data)/np.prod(m.shape)

0.0015038423791883377